In [2]:
%load_ext google.cloud.bigquery

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json'

In [8]:
%%bigquery
SELECT * FROM `apprenticeship-299321.sample_data.datatable`
LIMIT 10

DefaultCredentialsError: The file /Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json does not have a valid type. Type is None, expected one of ('authorized_user', 'service_account').

In [1]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    '/Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json',
)
df = pandas_gbq.read_gbq(sql, project_id="apprenticeship-299321", credentials=credentials)

ValueError: Service account info was not in the expected format, missing fields client_email, token_uri.

In [2]:
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/sqlservice.admin']
SERVICE_ACCOUNT_FILE = '/Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

ValueError: Service account info was not in the expected format, missing fields client_email, token_uri.

In [4]:
import httplib2

from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import run_flow

CLIENT_SECRET = '/Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json'
SCOPE = 'https://www.googleapis.com/auth/spreadsheets.readonly'
STORAGE = Storage('credentials.storage')

# Start the OAuth flow to retrieve credentials
def authorize_credentials():
# Fetch credentials from storage
    credentials = STORAGE.get()
# If the credentials doesn't exist in the storage location then run the flow
    if credentials is None or credentials.invalid:
        flow = flow_from_clientsecrets(CLIENT_SECRET, scope=SCOPE)
        http = httplib2.Http()
        credentials = run_flow(flow, STORAGE, http=http)
    return credentials
credentials = authorize_credentials()

/Users/colette/opt/anaconda3/lib/python3.8/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access credentials.storage: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))
usage: ipykernel_launcher.py [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/colette/Library/Jupyter/runtime/kernel-95e9b2dc-7371-4274-8f33-70481d1ee7e3.json


SystemExit: 2

/Users/colette/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


I think I keep trying to use methods which accept "service account key", rather than "OAuth client ID".

In [4]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [5]:
query = """
    SELECT * FROM `apprenticeship-299321.sample_data.datatable`
    LIMIT 10
"""
project_id = 'apprenticeship-299321'

In [6]:
import pandas as pd
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

,date,channelGrouping,deviceCategory,sessions,conversions
0,20180324,Email,mobile,32,0
1,20180518,Email,mobile,31,0
2,20180415,Email,mobile,29,0
3,20171206,Email,mobile,28,0
4,20180329,Email,mobile,27,0
5,20171227,Email,mobile,25,0
6,20180421,Email,mobile,24,0
7,20180127,Email,desktop,22,0
8,20170706,Email,mobile,20,0
9,20180519,Email,desktop,19,0


In [7]:
%%bigquery
SELECT * FROM `apprenticeship-299321.sample_data.datatable`
LIMIT 10

DefaultCredentialsError: The file /Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json does not have a valid type. Type is None, expected one of ('authorized_user', 'service_account').

In [8]:
testQuery1 = """
    SELECT * FROM `apprenticeship-299321.sample_data.datatable`
    LIMIT 100
"""
df = pd.read_gbq(testQuery1, project_id=project_id, dialect='standard', credentials=credentials)
df

,date,channelGrouping,deviceCategory,sessions,conversions
0,20180324,Email,mobile,32,0
1,20180518,Email,mobile,31,0
2,20180415,Email,mobile,29,0
3,20171206,Email,mobile,28,0
4,20180329,Email,mobile,27,0
...,...,...,...,...,...
95,20171222,Email,tablet,4,0
96,20180106,Email,mobile,4,0
97,20180107,Email,mobile,4,0
98,20180108,Email,desktop,4,0


## All data (chronological):
(TODO: add titles for other cells)

In [9]:
query0 = """
    SELECT * FROM `apprenticeship-299321.sample_data.datatable`
    ORDER BY date ASC
"""
df = pd.read_gbq(query0, project_id=project_id, dialect='standard', credentials=credentials)
df

,date,channelGrouping,deviceCategory,sessions,conversions
0,20170101,Social,tablet,10,0
1,20170101,(Other),mobile,14,0
2,20170101,Display,mobile,11,0
3,20170101,Retargeting,mobile,4,0
4,20170101,Retargeting,desktop,2,0
...,...,...,...,...,...
15842,20180520,Direct,mobile,1160,43
15843,20180520,Organic Search,tablet,989,64
15844,20180520,Aggregators,mobile,1795,71
15845,20180520,Organic Search,mobile,2840,91


In [10]:
# skip this because it's unnecessary...

# query1 = """
#     SELECT COUNT(*) FROM `apprenticeship-299321.sample_data.datatable`
# """
# df = pd.read_gbq(query1, project_id=project_id, dialect='standard', credentials=credentials)
# df

In [11]:
query2 = """
    SELECT deviceCategory, COUNT(*) AS deviceCategoryCount
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY deviceCategory
    ORDER BY deviceCategoryCount ASC
"""
df = pd.read_gbq(query2, project_id=project_id, dialect='standard', credentials=credentials)
df

,deviceCategory,deviceCategoryCount
0,tablet,5216
1,desktop,5263
2,mobile,5368


In [25]:
query3 = """
    SELECT channelGrouping, COUNT(*) AS channelGroupingCount
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY channelGrouping
    ORDER BY channelGroupingCount ASC
"""
df = pd.read_gbq(query3, project_id=project_id, dialect='standard', credentials=credentials)
df

,channelGrouping,channelGroupingCount
0,Newsletter,503
1,Partner referral,503
2,Email alerts,516
3,Retargeting,1085
4,Display,1216
5,Email,1475
6,(Other),1491
7,Social,1501
8,Aggregators,1501
9,Paid Search,1511


In [13]:
def runQuery(queryName):
    query = pd.read_gbq(queryName, project_id=project_id, dialect='standard', credentials=credentials)
    print(query) 

In [27]:
# um, what is this supposed to tell us?
# TODO: revisit this
query4 = """
    SELECT date, COUNT(*)
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY date
    ORDER BY date ASC
"""
runQuery(query4)

         date  f0_
0    20170101   30
1    20170102   30
2    20170103   30
3    20170104   30
4    20170105   30
..        ...  ...
500  20180516   33
501  20180517   33
502  20180518   34
503  20180519   34
504  20180520   34

[505 rows x 2 columns]


In [35]:
# this is rounded to 1 decimal place, while other data is rounded to 4 -
# this is because ideally I'd have used significant figures
query5 = """
    SELECT deviceCategory, ROUND(AVG(conversions), 1) as averageConversionsPerDeviceCategory
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY deviceCategory
    ORDER BY averageConversionsPerDeviceCategory
"""
runQuery(query5)

  deviceCategory  averageConversionsPerDeviceCategory
0         tablet                                 20.4
1         mobile                                 38.6
2        desktop                                 53.7


In [34]:
query6 = """
    SELECT deviceCategory, ROUND((AVG(conversions))/(AVG(sessions)), 4) as conversionRatePerDeviceCategory
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY deviceCategory
    ORDER BY conversionRatePerDeviceCategory
"""
runQuery(query6)

  deviceCategory  conversionRatePerDeviceCategory
0         mobile                           0.0261
1        desktop                           0.0430
2         tablet                           0.0551


In [33]:
query7 = """
    SELECT channelGrouping, ROUND((AVG(conversions))/(AVG(sessions)), 4) as conversionRatePerChannelGrouping
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY channelGrouping
    ORDER BY conversionRatePerChannelGrouping
"""
runQuery(query7)

     channelGrouping  conversionRatePerChannelGrouping
0             Social                            0.0287
1            (Other)                            0.0344
2         Newsletter                            0.0350
3        Paid Search                            0.0350
4             Direct                            0.0354
5              Email                            0.0357
6        Aggregators                            0.0358
7       Email alerts                            0.0358
8     Organic Search                            0.0368
9           Referral                            0.0370
10       Retargeting                            0.0387
11  Partner referral                            0.0399
12           Display                            0.0444


In [37]:
query8 = """
    SELECT date, ROUND((AVG(conversions))/(AVG(sessions)), 4) as conversionRate
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY date
    ORDER BY date ASC
"""
runQuery(query8)

         date  conversionRate
0    20170101          0.0313
1    20170102          0.0308
2    20170103          0.0372
3    20170104          0.0317
4    20170105          0.0400
..        ...             ...
500  20180516          0.0399
501  20180517          0.0345
502  20180518          0.0390
503  20180519          0.0342
504  20180520          0.0382

[505 rows x 2 columns]


In [38]:
query9 = """
    SELECT date, SUM(sessions) as totalSessions
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY date
    ORDER BY date ASC
"""
runQuery(query9)

         date  totalSessions
0    20170101          17367
1    20170102          27842
2    20170103          42792
3    20170104          42956
4    20170105          43019
..        ...            ...
500  20180516          32012
501  20180517          26716
502  20180518          22831
503  20180519          14260
504  20180520          17993

[505 rows x 2 columns]


In [15]:
%load_ext sql

In [17]:
# %%bigquery
query10 = """
    SELECT date, SUM(conversions) as totalConversions
    FROM `apprenticeship-299321.sample_data.datatable`
    GROUP BY date
    ORDER BY date ASC
"""
runQuery(query10)
# result10 = %sql $query10

DefaultCredentialsError: The file /Users/colette/Documents/GitHub/More-BigQuery-with-Python/ApprenticeshipOAuth.json does not have a valid type. Type is None, expected one of ('authorized_user', 'service_account').

In [40]:
# %matplotlib inline